# **UTS PROSAINDATA**


## **KLASIFIKASI TEXT ABSTRAK DENGAN KNN MENGGUNAKAN REPRESENTASI TEXT TF-IDF DENGAN REDUKSI DIMENSI PCA**







In [2]:
import numpy as np
import pandas as pd

## **Load Data**

In [3]:
df = pd.read_csv("/content/drive/MyDrive/prosaindata/tugas/dataset-baru.csv", encoding = 'unicode_escape', engine ='python')

In [4]:
df.drop(columns=['Unnamed: 0'])

,Abstrak,Kategori
0,Sistem informasi akademik (SIAKAD) merupaka...,RPL
1,Berjalannya koneksi jaringan komputer dengan l...,RPL
2,Web server adalah sebuah perangkat lunak serve...,RPL
3,Penjadwalan kuliah di Perguruan Tinggi me...,KOMPUTASI
4,Seiring perkembangan teknologi yang ada diduni...,RPL
...,...,...
800,Investasi saham selama ini memiliki resiko ker...,KOMPUTASI
801,Information Retrieval (IR) merupakan pengambil...,KOMPUTASI
802,Klasifikasi citra merupakan proses pengelompok...,KOMPUTASI
803,Identifikasi atribut pejalan kaki merupakan sa...,KOMPUTASI


## **Normalisasi Text**

**Menginstall Library yang dibutuhkan untuk Preprocessing**

In [5]:
!pip install nltk
!pip install sastrawi
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 9.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 KB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.1 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16321 sha256=ae5945c54694f3e1c6b8e0204441c00c90637e4003a5d73ed650d7c18cbf86c5
  Stored in directory: /root/.cache/pip/wheels/2b/5e/f2/3931524f702ffd03309e96d35ee2fbf9c61c27377511ee8d4c
Successfully built swifter


In [6]:
import nltk
from nltk.corpus import stopwords
import re
import swifter

In [7]:
text = df['Abstrak']
Y = df['Kategori'].values
text

0      Sistem  informasi  akademik  (SIAKAD) merupaka...
1      Berjalannya koneksi jaringan komputer dengan l...
2      Web server adalah sebuah perangkat lunak serve...
3      Penjadwalan  kuliah  di  Perguruan  Tinggi  me...
4      Seiring perkembangan teknologi yang ada diduni...
                             ...                        
800    Investasi saham selama ini memiliki resiko ker...
801    Information Retrieval (IR) merupakan pengambil...
802    Klasifikasi citra merupakan proses pengelompok...
803    Identifikasi atribut pejalan kaki merupakan sa...
804    Topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 805, dtype: object

**Case Folding**

In [8]:
def case_folding(comment):
    comment = comment.lower()
    return comment
text = text.apply(case_folding)
text

0      sistem  informasi  akademik  (siakad) merupaka...
1      berjalannya koneksi jaringan komputer dengan l...
2      web server adalah sebuah perangkat lunak serve...
3      penjadwalan  kuliah  di  perguruan  tinggi  me...
4      seiring perkembangan teknologi yang ada diduni...
                             ...                        
800    investasi saham selama ini memiliki resiko ker...
801    information retrieval (ir) merupakan pengambil...
802    klasifikasi citra merupakan proses pengelompok...
803    identifikasi atribut pejalan kaki merupakan sa...
804    topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 805, dtype: object

**Menghilangkan link/url**

In [9]:
def linkNormalize(text):
    text = re.sub(r"\s—\s", "", text)
    text = re.sub(r"http\S+", "", text)
    return text
text = text.apply(linkNormalize)
text

0      sistem  informasi  akademik  (siakad) merupaka...
1      berjalannya koneksi jaringan komputer dengan l...
2      web server adalah sebuah perangkat lunak serve...
3      penjadwalan  kuliah  di  perguruan  tinggi  me...
4      seiring perkembangan teknologi yang ada diduni...
                             ...                        
800    investasi saham selama ini memiliki resiko ker...
801    information retrieval (ir) merupakan pengambil...
802    klasifikasi citra merupakan proses pengelompok...
803    identifikasi atribut pejalan kaki merupakan sa...
804    topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 805, dtype: object

**Menghilangkan karakter tertentu**

In [10]:
symbols = ",!\"#$%&()*+-.…/:;<=>?@[\]^_`{|}~\n"
def cleaning_text(comment):
    for i in symbols:
        comment = np.char.replace(comment, i, ' ')
    return comment
text = text.apply(cleaning_text)
text

0      sistem  informasi  akademik   siakad  merupaka...
1      berjalannya koneksi jaringan komputer dengan l...
2      web server adalah sebuah perangkat lunak serve...
3      penjadwalan  kuliah  di  perguruan  tinggi  me...
4      seiring perkembangan teknologi yang ada diduni...
                             ...                        
800    investasi saham selama ini memiliki resiko ker...
801    information retrieval  ir  merupakan pengambil...
802    klasifikasi citra merupakan proses pengelompok...
803    identifikasi atribut pejalan kaki merupakan sa...
804    topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 805, dtype: object

**Menghilangkan Stopwords menggunakan library nltk**

In [11]:
nltk.download('stopwords')
list_stopwords = stopwords.words('indonesian')
def stopword_removal(comment):
    comment = comment.split()
    data = [word for word in comment if word not in list_stopwords]
    kalimat = ""
    for i in data:
      kalimat += i
      kalimat += " "
    return kalimat

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
text = text.apply(stopword_removal)

**Memperbaiki kata yang yang tidak baku menggunakan library sastrawi**

In [13]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stemmingNormalize(comment):
    comment = comment.split()
    data = []
    for term in comment:
        term = stemmer.stem(term)
        data.append(term)
    kalimat = ""
    for i in data:
      kalimat += i
      kalimat += " "
    return kalimat
text = text.swifter.apply(stemmingNormalize)
text

Pandas Apply:   0%|          | 0/805 [00:00<?, ?it/s]

0      sistem informasi akademik siakad sistem inform...
1      jalan koneksi jaring komputer lancar ganggu ha...
2      web server perangkat lunak server fungsi terim...
3      jadwal kuliah guru kompleks masalah variabel t...
4      iring kembang teknologi dunia muncul teknologi...
                             ...                        
800    investasi saham milik resiko rugi gera harga s...
801    information retrieval ir ambil informasi simpa...
802    klasifikasi citra proses kelompok piksel citra...
803    identifikasi atribut pejal kaki salah teliti k...
804    topik deteksi objek tarik perhati kembang tekn...
Name: Abstrak, Length: 805, dtype: object

## **Perhitungan menggunakan TF-IDF scikit learn**

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_separate = tfidf_vectorizer.fit_transform(text)

df_tfidf = pd.DataFrame(
    tfidf_separate.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=df.index
)
X = df_tfidf.values
df_tfidf

,00,000,00001,0001,000505341,000506436,000792353,00086,00089,001,...,zara,zat,zcz,zf,zona,zone,zoning,zoom,zucara,zungu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **Membagi Data Training dan Data Test**
data traning sebesar 80%
data test sebesar 20%

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( 
    X, Y, test_size = 0.2, random_state = 100)
print("Jumlah Data training : ", len(X_train))
print("Jumlah Data test : ", len(X_test))

Jumlah Data training :  644
Jumlah Data test :  161


## **Menghitung Akurasi menggunakan Model KNN dengan reduksi dimensi(PCA)**

In [32]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 10)

X_train_2 = pca.fit_transform(X_train)
X_test_2 = pca.transform(X_test)

pca.explained_variance_ratio_

array([0.02192352, 0.01584224, 0.01479795, 0.01248874, 0.01058662,
       0.00932052, 0.00844153, 0.00801767, 0.00778969, 0.00727099])

In [33]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train_2, y_train)

KNeighborsClassifier()

In [34]:
from sklearn.metrics import accuracy_score
y_pred = classifier.predict(X_test_2)
print (" KNN Accuracy : ",
    accuracy_score(y_test,y_pred)*100)

 KNN Accuracy :  81.98757763975155


## **Akurasi menggunakan Model KNN tanpa reduksi dimensi(PCA)**

In [35]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [36]:
from sklearn.metrics import accuracy_score
y_pred = classifier.predict(X_test)
print (" KNN Accuracy : ",
    accuracy_score(y_test,y_pred)*100)

 KNN Accuracy :  83.85093167701864
